<a href="https://colab.research.google.com/github/tarunbalajiks/Speech-Emotion-Recognition/blob/main/DeployWebApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install pyngrok

In [39]:
!ngrok config add-authtoken AUTH_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [62]:
import keras
import librosa
import numpy as np
from keras.layers import Input, Conv1D, Flatten, Dense, Dropout, Activation, MaxPooling1D
from keras.models import Model
from flask import Flask, request, redirect, render_template_string, send_from_directory
from werkzeug.utils import secure_filename
import os
from io import BytesIO
import base64
import os
from pyngrok import ngrok

In [63]:
''' Class to Make Predicitions '''

class LivePredictions:
    def __init__(self):
        self.path = "/content/Emotion_Voice_Detection_Model.h5"

        input_layer = Input(shape=(40, 1))
        x = Conv1D(64, 5, padding='same')(input_layer)
        x = Activation('relu')(x)
        x = Dropout(0.2)(x)
        x = Flatten()(x)
        x = Dense(8)(x)
        output_layer = Activation('softmax')(x)
        self.model = Model(inputs=input_layer, outputs=output_layer)
        self.model.load_weights(self.path)

    @staticmethod
    def convert_class_to_emotion(pred):
        label_conversion = {'0': 'neutral',
                            '1': 'calm',
                            '2': 'happy',
                            '3': 'sad',
                            '4': 'angry',
                            '5': 'fearful',
                            '6': 'disgust',
                            '7': 'surprised'}

        for key, value in label_conversion.items():
            if int(key) == pred:
                label = value

        return label

    def make_predictions(self, file):
        data, sampling_rate = librosa.load(file)
        mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=40).T, axis=0)
        x = np.expand_dims(mfccs, axis=0)
        predictions = self.model.predict(x, verbose=0)
        predicted_class = np.argmax(predictions)

        return self.convert_class_to_emotion(predicted_class)

In [64]:
''' Check if model is working correctly '''

predection = LivePredictions()
predection.model.summary()
wrong_classification_path = '/content/03-01-01-01-01-02-05.wav'
correct_classification_path = '/content/10-16-07-29-82-30-63.wav'

correct = predection.make_predictions(file=correct_classification_path)
print(f"Correct Emotion Is {correct}")
wrong = predection.make_predictions(file=wrong_classification_path)
print(f"Wrong Emotion Is {wrong}")

Model: "functional_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_28 (InputLayer)          │ (None, 40, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_31 (Conv1D)                   │ (None, 40, 64)              │             384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_59 (Activation)           │ (None, 40, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_31 (Dropout)                 │ (None, 40, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_28 (Flatten)                 │ (None, 2560)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 8)                   │          20,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_60 (Activation)           │ (None, 8)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,872 (81.53 KB)

 Trainable params: 20,872 (81.53 KB)

 Non-trainable params: 0 (0.00 B)

Correct Emotion Is disgust
Wrong Emotion Is calm


In [66]:
from flask import Flask, request, redirect, render_template_string, send_from_directory
from werkzeug.utils import secure_filename
import os
from io import BytesIO
import base64
import os
from pyngrok import ngrok

# Flask setup
app = Flask(__name__)

app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['ALLOWED_EXTENSIONS'] = {'wav', 'mp3'}


if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']


live_prediction = LivePredictions()

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)

            predicted_emotion = live_prediction.make_predictions(file=filepath)
            return render_template_string(TEMPLATE_MATCH_FOUND, emotion=predicted_emotion, audio_filename=filename)

    return render_template_string(TEMPLATE_UPLOAD)

TEMPLATE_UPLOAD = '''
<!doctype html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Speech Emotion Recognition</title>
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
    <style>
        body {
            background: linear-gradient(135deg, rgba(238, 174, 202, 1) 0%, rgba(148, 187, 233, 1) 100%);
            font-family: 'Roboto', sans-serif;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            height: 100vh;
            margin: 0;
            color: #333;
        }
        h1 {
            font-size: 2.5em;
            margin-bottom: 10px;
            font-weight: bold;
        }
        p {
            font-size: 1.2em;
            text-align: center;
            margin: 0 0 30px;
        }
        .upload-container {
            background: white;
            border-radius: 10px;
            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.1);
            padding: 30px;
            text-align: center;
            width: 90%;
            max-width: 400px;
        }
        .upload-btn {
            background-color: #6c63ff;
            border: none;
            border-radius: 5px;
            color: white;
            padding: 15px 35px;
            font-size: 1em;
            cursor: pointer;
            transition: background 0.3s;
        }
        .upload-btn:hover {
            background-color: #5753d1;
        }
        .upload-area {
            border: 2px dashed #6c63ff;
            border-radius: 10px;
            padding: 30px;
            margin: 10px 0;
            transition: background 0.3s;
        }
        .upload-area:hover {
            background-color: #f3f3f3;
        }
        input[type="file"] {
            display: none; /* Hide the default file input */
        }
        label {
            margin: 10px 0;
            cursor: pointer;
            color: #6c63ff;
        }
        #fileFeedback {
            display: none;
            color: #6c63ff;
            margin-top: 15px;
        }
        .loading {
            display: none;
            font-weight: bold;
            margin-top: 15px;
        }
    </style>
    <script>
        function updateUploadStatus(event) {
            const feedback = document.getElementById('fileFeedback');
            const loading = document.getElementById('loadingMessage');

            if (event.target.files.length > 0) {
                feedback.textContent = "File selected: " + event.target.files[0].name;
                feedback.style.display = "block";
                loading.style.display = "none";
            } else {
                feedback.style.display = "none";
                loading.style.display = "none";
            }
        }

        function showLoading() {
            const loading = document.getElementById('loadingMessage');
            loading.style.display = 'block';
        }
    </script>
</head>
<body>
    <h1>Speech Emotion Recognition</h1>
    <p>Upload an audio file to analyze the emotional content of speech. Our AI will detect various emotions and their intensities.</p>
    <div class="upload-container">
        <form method="post" enctype="multipart/form-data" onsubmit="showLoading();">
            <div class="upload-area">
                <label for="file">Upload audio file</label>
                <input type="file" name="file" accept="audio/*" id="file" required onchange="updateUploadStatus(event);">
                <p>Drag and drop or click to select</p>
                <div id="fileFeedback"></div>
                <div id="loadingMessage" class="loading">Loading... Please wait.</div>
            </div>
            <button class="upload-btn" type="submit">Select File</button>
        </form>
    </div>
</body>
</html>
'''

TEMPLATE_MATCH_FOUND = '''
<!doctype html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Emotion Recognized</title>
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
    <style>
        body {
            background: linear-gradient(135deg, rgba(238, 174, 202, 1) 0%, rgba(148, 187, 233, 1) 100%);
            font-family: 'Roboto', sans-serif;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            height: 100vh;
            margin: 0;
            color: #333;
        }
        h1 {
            font-size: 2.5em;
            margin-bottom: 10px;
            font-weight: bold;
        }
        .result-container {
            background: white;
            border-radius: 10px;
            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.1);
            padding: 20px;
            text-align: center;
            width: 90%;
            max-width: 400px;
        }
        audio {
            margin-top: 20px;
            width: 100%;
        }
        .try-again-btn {
            background: #ff4c4c;
            border: none;
            padding: 10px 20px;
            border-radius: 5px;
            color: white;
            font-size: 1em;
            cursor: pointer;
            margin-top: 20px;
            transition: background 0.3s;
        }
        .try-again-btn:hover {
            background: #cc0000;
        }
    </style>
</head>
<body>
    <h1>Emotion Recognized: {{ emotion }}</h1>
    <div class="result-container">
        <audio controls>
            <source src="{{ url_for('uploaded_file', filename=audio_filename) }}" type="audio/wav">
            Your browser does not support the audio tag.
        </audio>
        <form method="get" action="/">
            <button class="try-again-btn" type="submit">Try Again</button>
        </form>
    </div>
</body>
</html>
'''

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

# Run the Flask app
port = 5000
app.run(host='0.0.0.0', port=port)
public_url = ngrok.connect(port)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")
app.run(host='0.0.0.0', port=port)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


 * ngrok tunnel "NgrokTunnel: "https://52ca-35-194-9-120.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Dec/2024 06:03:21] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2024 06:03:22] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2024 06:03:28] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2024 06:03:28] "GET /uploads/YAF_witch_sad.mp3 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2024 06:03:30] "GET /favicon.ico HTTP/1.1" 404 -
